In [1]:
from simclr import SimCLR
from data import create_cifar_train_loader
import torch
import torch.nn as nn
from models import Cifar10_SimCLR_Model

In [4]:
# load data
dataloader = create_cifar_train_loader()

# intialize model
model = Cifar10_SimCLR_Model()

# train model
trainer = SimCLR(model)
trainer.train(dataloader, epochs=1)

Files already downloaded and verified


Epoch: 1/1:   0%|          | 0/391 [00:10<?, ?it/s]


AttributeError: 'ResNet' object has no attribute 'maxpool'

In [7]:
from torchvision.models.resnet import resnet50

encoder = resnet50(pretrained=False)
encoder.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
del encoder.maxpool
del encoder.fc

print(*encoder.modules())

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, trac